In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
movies=pd.read_csv('Movies.csv')

In [4]:
movies.head(5)

,id,original_language,original_title,overview,popularity,poster_path,release_date,runtime,title,vote_average,vote_count,production_company
0,862,English,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,81,Toy Story,7.7,5415.0,Pixar Animation Studios
1,8844,English,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,104,Jumanji,6.9,2413.0,TriStar Pictures
2,11862,English,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,1995-02-10,106,Father of the Bride Part II,5.7,173.0,Sandollar Productions
3,949,English,Heat,"Obsessive master thief, Neil McCauley leads a ...",17.924927,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,1995-12-15,170,Heat,7.7,1886.0,Regency Enterprises
4,9091,English,Sudden Death,International action superstar Jean Claude Van...,5.231580,/eoWvKD60lT95Ss1MYNgVExpo5iU.jpg,1995-12-22,106,Sudden Death,5.5,174.0,Universal Pictures


In [5]:
movies = movies[['id','overview','title','original_language','production_company']]

In [6]:
movies.head(3)

,id,overview,title,original_language,production_company
0,862,"Led by Woody, Andy's toys live happily in his ...",Toy Story,English,Pixar Animation Studios
1,8844,When siblings Judy and Peter discover an encha...,Jumanji,English,TriStar Pictures
2,11862,Just when George Banks has recovered from his ...,Father of the Bride Part II,English,Sandollar Productions


In [7]:
# # remove spaces between words 

# movies['genres'] = movies['overview'].apply(lambda x:[i.replace(" ","") for i in x])
# movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
# movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
# movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [8]:
movies.head(3)

,id,overview,title,original_language,production_company
0,862,"Led by Woody, Andy's toys live happily in his ...",Toy Story,English,Pixar Animation Studios
1,8844,When siblings Judy and Peter discover an encha...,Jumanji,English,TriStar Pictures
2,11862,Just when George Banks has recovered from his ...,Father of the Bride Part II,English,Sandollar Productions


In [9]:
movies['features'] = movies['overview'] + movies['title'] + movies['original_language'] + movies['original_language']
 

In [10]:
movies['features'][0]

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.Toy StoryEnglishEnglish"

In [11]:
#lower casing all the alphabets in the tags column
movies['features'] = movies['features'].apply(lambda x:x.lower()) 

In [12]:
# Apply Steming to remove similarities/duplications in words list
# import nltk
# from nltk.stem.porter import PorterStemmer
# ps = PorterStemmer()

# def xStem(txt):
#     y = []
#     for x in txt.split():
#         y.append(ps.stem(x))
#     return " ".join(y)
# movies['features'] = movies['features'].apply(xStem)   

In [13]:
#apply lemmatization to remove similarities/duplications in words list
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemm(txt):
    y = []
    for x in txt.split():
        y.append(lemmatizer.lemmatize(x))
    return " ".join(y)
movies['features'] = movies['features'].apply(lemm) 

In [14]:
movies['features'][0]

"led by woody, andy's toy live happily in his room until andy's birthday brings buzz lightyear onto the scene. afraid of losing his place in andy's heart, woody plot against buzz. but when circumstance separate buzz and woody from their owner, the duo eventually learns to put aside their differences.toy storyenglishenglish"

In [15]:
#convert text to matrix
from sklearn.feature_extraction.text import TfidfVectorizer 
tf=TfidfVectorizer(stop_words="english")
tf_vector= tf.fit_transform(movies['features']).toarray()

In [16]:
similarity = cosine_similarity(tf_vector)

In [17]:
pd.DataFrame(similarity,index=movies['title'],columns=movies['title'])

title,Toy Story,Jumanji,Father of the Bride Part II,Heat,Sudden Death,GoldenEye,The American President,Dracula: Dead and Loving It,Balto,Casino,...,2:22,The Dark Tower,The Emoji Movie,Wind River,Shot Caller,Brice 3,Girls Trip,Descendants 2,"Good Guys Go to Heaven, Bad Guys Go to Pattaya",The Visitors: Bastille Day
title,,,,,,,,,,,,,,,,,,,,,
Toy Story,1.000000,0.017134,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
Jumanji,0.017134,1.000000,0.000000,0.045522,0.092907,0.000000,0.006508,0.000000,0.000000,0.000000,...,0.000000,0.016932,0.000000,0.071518,0.0,0.015966,0.000000,0.000000,0.005241,0.0
Father of the Bride Part II,0.000000,0.000000,1.000000,0.000000,0.038857,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.005886,0.000000,0.000000,0.0,0.008219,0.000000,0.027433,0.011037,0.0
Heat,0.000000,0.045522,0.000000,1.000000,0.044260,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.011046,0.013941,0.000000,0.026458,0.0,0.018232,0.000000,0.000000,0.028595,0.0
Sudden Death,0.000000,0.092907,0.038857,0.044260,1.000000,0.000000,0.000000,0.077348,0.000000,0.000000,...,0.000000,0.000000,0.016376,0.067192,0.0,0.000000,0.000000,0.041466,0.020344,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Brice 3,0.000000,0.015966,0.008219,0.018232,0.000000,0.000000,0.044605,0.000000,0.000000,0.000000,...,0.000000,0.022520,0.000000,0.000000,0.0,1.000000,0.039594,0.009814,0.040130,0.0
Girls Trip,0.000000,0.000000,0.000000,0.000000,0.000000,0.040235,0.032277,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.039594,1.000000,0.000000,0.000000,0.0
Descendants 2,0.000000,0.000000,0.027433,0.000000,0.041466,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.007028,0.000000,0.000000,0.0,0.009814,0.000000,1.000000,0.000000,0.0


In [18]:
distances = similarity[32] ## Similarites for the movie 
sorted(distances,reverse=True)[0:10]

[1.0000000000000004,
 0.11856709181849777,
 0.1032471151397729,
 0.09698942762345633,
 0.09362331934307291,
 0.0903520045318305,
 0.08831990359202652,
 0.0881190536000001,
 0.08439746948812699,
 0.08353217349066382]

In [19]:
def recommend(movie):
    movie_index = movies[movies['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True, key=lambda x:x[1])[1:16]
    mov=[]
    id=[]
    scores=[]
    for i in movies_list:
        mov.append(movies.iloc[i[0]].title)
        id.append(movies.iloc[i[0]].id)
        scores.append(i[1])
    dic={'id':id,'title':mov,'Similarity Score':scores}
    return pd.DataFrame(dic) 

How the recommender function works ?

In [20]:
# we first get the index of the movie for example Alice in Wonderland index is 32
movie_index = movies[movies['title'] == 'Jumanji'].index[0]
movie_index

1

In [21]:
# we get the similarities for the movie
distances = similarity[1] 
# we sort the similarities in descending order
sorted(distances,reverse=True)[0:10]

[0.9999999999999998,
 0.1777818849872453,
 0.13302164995309596,
 0.13155022893013105,
 0.12947290022814517,
 0.12667383583516328,
 0.12635666419523314,
 0.11994373499137767,
 0.11526788265311447,
 0.1123999492296163]

In [22]:
#then we get the top 3 movies and their Index and similarity score
num=3
movies_list = sorted(list(enumerate(distances)),reverse=True, key=lambda x:x[1])[1:num+1]
movies_list

[(3427, 0.1777818849872453),
 (1604, 0.13302164995309596),
 (3416, 0.13155022893013105)]

In [23]:
# finally we access them using Index
for i in movies_list:
 print(movies.iloc[i[0]].title)
 print(movies.iloc[i[0]].id)

Indie Game: The Movie
80215
Spy Kids 3-D: Game Over
12279
Wreck-It Ralph
82690


In [24]:
#generate random title from the list of movies
import random
def random_title():
    return random.choice(movies['title'])

movie_name=random_title()
movie_name

'Ju-on: The Grudge'

In [25]:
popular_movies = recommend(movie_name) 
popular_movies

,id,title,Similarity Score
0,1970,The Grudge,0.160788
1,10731,The Client,0.121596
2,72571,Paranormal Activity 3,0.116122
3,10727,Darkness Falls,0.105642
4,8271,Disturbia,0.105061
5,9100,The Craft,0.096213
6,2114,Final Fantasy: The Spirits Within,0.092909
7,4970,Gothika,0.082881
8,9378,Thir13en Ghosts,0.082562
9,91586,Insidious: Chapter 2,0.081968


In [26]:
import requests
from IPython.display import Image, HTML, display
def movie_display(popular_movies):
 getList_name = {}
 for x, xRows in popular_movies.iterrows():

    getResponse = requests.get('https://api.themoviedb.org/3/movie/{}?api_key=c0bda0be71f7815fd6ba2eb5f5c86fd8'.format(xRows['id']) ) # every movie has a unique ID 
    getData = getResponse.json() # we request the data from the API and convert it to json
    
    # a bug fixed because sometimes there are is no poster so it returrns error
    if getData['poster_path']==None:
     continue
    else:   
     getPath = "http://image.tmdb.org/t/p/w500" + getData['poster_path']    # get the path of the poster
     getList_name[xRows['title']] = getPath
 
 
 display(HTML(f"""  < <div style="font-size:24px; font-weight:Bold; color:#fff; text-align:center; padding-top:8px; height:12%; width: 100%; border:1px solid #ccc; border-radius:10px; margin-top:10px; background-color:#FA1A1A;">{movie_name}</div> """))   
 # in here he loops on the number of movies to be recommended which is in num_recommend variable in recommend() function
 for i in range(0,popular_movies.shape[0],5):
  

  display( HTML(f"""
                    <table>
                    <tr>
                        <td><img src={list(getList_name.values())[i]} style='border-radius:10px; height:400px; width:575px; border:1px solid #999;'></td>
                        <td><img src={list(getList_name.values())[i+1]} style='border-radius:10px; height:400px; width:575px; border:1px solid #999;'></td>
                        <td><img src={list(getList_name.values())[i+2]} style='border-radius:10px; height:400px; width:575px; border:1px solid #999;'></td>
                        <td><img src={list(getList_name.values())[i+3]} style='border-radius:10px; height:400px; width:575px; border:1px solid #999;'></td>
                        <td><img src={list(getList_name.values())[i+4]} style='border-radius:10px; height:400px; width:575px; border:1px solid #999;'></td>
                        
                    </tr>
                  
                        <td><div style="height:60px; padding-top:15px; text-align:center; font-size:14px; font-weight:bold; border:1px solid #ccc; border-radius:10px;">{list(getList_name.keys())[i+0]}</div></td>
                        <td><div style="height:60px; padding-top:15px; text-align:center; font-size:14px; font-weight:bold; border:1px solid #ccc; border-radius:10px;">{list(getList_name.keys())[i+1]}</div></td>
                        <td><div style="height:60px; padding-top:15px; text-align:center; font-size:14px; font-weight:bold; border:1px solid #ccc; border-radius:10px;">{list(getList_name.keys())[i+2]}</div></td>
                        <td><div style="height:60px; padding-top:15px; text-align:center; font-size:14px; font-weight:bold; border:1px solid #ccc; border-radius:10px;">{list(getList_name.keys())[i+3]}</div></td>
                        <td><div style="height:60px; padding-top:15px; text-align:center; font-size:14px; font-weight:bold; border:1px solid #ccc; border-radius:10px;">{list(getList_name.keys())[i+4]}</div></td>
                    </tr>
                    
                </table>"""))

In [27]:
movie_display(recommend(movie_name))

,,,,,The Grudge,The Client,Paranormal Activity 3,Darkness Falls,Disturbia


,,,,,The Craft,Final Fantasy: The Spirits Within,Gothika,Thir13en Ghosts,Insidious: Chapter 2


,,,,,Marnie,Premium Rush,My Neighbor Totoro,The Unborn,Rent


In [36]:
# saving the the similaries and the cleaned data
import pickle 
pickle_out = open("tfidf.pkl", mode = "wb") 
pickle.dump(similarity, pickle_out)

In [29]:
# import bz2file as bz2
# with bz2.BZ2File( 'Tfidf'+ '.pbz2', 'wb') as f:
#  pickle.dump(similarity, f)

In [30]:
# def decompress_pickle(file):

#     data = bz2.BZ2File(file, 'rb')
#     data = pickle.load(data)
#     return data
# model = decompress_pickle('Tfidf.pbz2')

In [37]:
infile = open('tfidf.pkl','rb')
new_dict = pickle.load(infile)
infile.close()

EOFError: Ran out of input

In [35]:
print(infile)

<_io.BufferedReader name='tfidf.pkl'>
